In [64]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.environment_simulator.models import SimulatedCommodityBuffer
from apps.environment_simulator.sevice_layer.commodity_simulator import CommoditySimulator
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf

In [80]:
np.random.seed(0)
for i in range(1000):
    starting_latest_time_step = SimulatedCommodityBuffer.objects.first().captured_at
    # starting_latest_time_step = pytz.utc.localize(datetime.strptime(str(starting_latest_time_step), '%Y-%m-%d %H:%M:%S'))
    starting_next_time_step = starting_latest_time_step + relativedelta(hours= 2, minutes=30)

    latest_time_step = starting_latest_time_step
    next_time_step = starting_next_time_step
    cmmdty_smltr = CommoditySimulator(latest_time_step, next_time_step)
    cmmdty_smltr.vary_commodity_prices()

In [12]:
set = 8000.0
x = 7949.538159146002 + set*(
    0.008331516517697953 
    - 0.004478571188740903 
    + 0.01722214354311691 
    + 0.013152424016901289
    - 0.01732241508539263
)
print(x)
print(3.25/3.3166940076617384)

8084.778941574663
0.9798914197367403


In [81]:
cf.go_offline()
cf.set_config_file(world_readable=True, theme="white")

buffer = SimulatedCommodityBuffer.objects.select_related("commodity").order_by("captured_at").all()

price_variations = {}
for snapshot in buffer:
    if snapshot.commodity.name not in price_variations:
        price_variations[snapshot.commodity.name] = []
    price_variations[snapshot.commodity.name].append(snapshot.price_snapshot)

# print(price_variations)
for cmmdty_name in price_variations:
    price_snaps = price_variations[cmmdty_name]
    price_array = np.array(np.absolute(price_snaps))
    max_price = max(price_array)
    normalized_snaps = price_array/max_price
    price_variations[cmmdty_name] = normalized_snaps

    

commodity_names = list(price_variations.keys())


df_regret_comparisons = pd.DataFrame(price_variations)
df_regret_comparisons.iplot(
    xTitle='time_steps',
    yTitle='Normalized Prices'
)